# WeiRules (Yahoo images classification)

This is an example use of the neuro fuzzy model presented in:

1. "Pitsikalis, M., Do, TT., Lisitsa, A., Luo, S. (2021). Logic Rules Meet Deep Learning: A Novel Approach for Ship Type Classification. In: RuleML+RR 2021. https://doi.org/10.1007/978-3-030-91167-6_14" and 
2. "Logic Rules Meet Deep Learning: A Novel Approach for Ship Type Classification (Extended Abstract). Manolis Pitsikalis, Thanh-Toan Do, Alexei Lisitsa, Shan Luo. IJCAI Artificial Intelligence Sister Conferences Best Papers. Pages 5324-5328. https://doi.org/10.24963/ijcai.2022/744"

Since the images of the original maritime dataset are propriatery, we release the source code and an example application on a dataset of similar nature. 

Disclaimer: in what follows we will not use any hyperparameter tuning for any of the models, instead we will use the default parameters.

In [1]:
import frcnn as fw
import weirules as wr
import w_utils as wutils

import torch
from torch.utils import data
import torchvision.transforms as transforms
import torchvision

import pandas as pd
import numpy as np
import glob
import os
import pickle
import matplotlib.pyplot as plt


from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import locale
import warnings
warnings.filterwarnings('ignore')

## Data loading & preparation

For this experiment we are using the Yahoo test dataset that can be found here https://vision.cs.uiuc.edu/attributes/. The dataset contains images accompanied with bounding boxes and tabular data.

### Data load

In [2]:
attribute_names=[]
with open('sample/yahoo/attribute_data/attribute_names.txt','r') as inp:
    for line in inp:
        attribute_names.append(line.rstrip())
attribute_names_characteristics=attribute_names[6:]

In [3]:
box_columns=['xmin','ymin','xmax','ymax']
columns= ['img','class']+box_columns+attribute_names
yahoo_images=pd.read_csv('sample/yahoo/attribute_data/ayahoo_test.txt',delimiter=' ',header=None,names=columns)
yahoo_images=yahoo_images.drop_duplicates(subset=['img'])

In [4]:
yahoo_images

,img,class,xmin,ymin,xmax,ymax,2D Boxy,3D Boxy,Round,Vert Cyl,...,Wood,Cloth,Furry,Glass,Feather,Wool,Clear,Shiny,Vegetation,Leather
0,donkey_1.jpg,donkey,14,21,54,72,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,donkey_100.jpg,donkey,14,98,125,214,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,donkey_102.jpg,donkey,332,29,614,438,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,donkey_111.jpg,donkey,23,52,295,244,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,donkey_113.jpg,donkey,32,158,159,286,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2639,carriage_921.jpg,carriage,147,111,314,184,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2640,carriage_922.jpg,carriage,70,82,229,205,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2641,carriage_923.jpg,carriage,5,173,256,348,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2642,carriage_937.jpg,carriage,9,4,189,147,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Two images contained invalid bounding boxes as the xmin was equal to xmax (see below). The two invalid bounding boxes were corrected manually.

In [5]:
#yahoo_images[yahoo_images['xmin'] >= yahoo_images['xmax']]

### Label encoder
Use of label encoder to transform the n classes to the range [0,n-1]

In [6]:
le = preprocessing.LabelEncoder()
yahoo_images['class']=le.fit_transform(yahoo_images['class'])
classes=yahoo_images['class'].unique()
classes.sort()

### Training - testing set split
The test size = 0.49 for the following experiment was set to 0.49

In [7]:
from sklearn.model_selection import train_test_split
y=yahoo_images['class']
x_train, x_test, y_train, y_test = train_test_split(yahoo_images, y, test_size=0.49, random_state=42, stratify=y)

In [8]:
attributes_train=x_train[attribute_names_characteristics]
attributes_test=x_test[attribute_names_characteristics]

class_labels_train=y_train
class_labels_test=y_test

image_paths_train=x_train['img']
image_paths_test=x_test['img']

boxes_train=x_train[box_columns]
boxes_test=x_test[box_columns]

## Training

The steps for training WeiRules are the following:
  1. Fit a decision tree (or trees) on the tabular data, extract the rules, and create the network with the fuzzified rules.
  2. Train Faster R-CNN on the images with the bounding boxes and the labels
  3. Extract deep features corresponding to the highest score yielding box
  4. Train WeiRules using the extracted deep features and the tabular data corresponding to the image


### Rule extraction and fuzzification

Create decision trees, extract the rules, and then create the neuro-fuzzy the network. `fit_tree` in this case, will learn a separate tree for each class by substituting the remaining classes with -1, therefore treating the problem locally as a binary classification task. The result here would be a set of rules each corresponding to each class. Setting `find_best_tree=true` will search for the tree depth up to `max_depth` that yields the highest macro F1-score on the training set. 

Single tree is also possible by setting `forest=False`. 

Uncomment the code below if you want to recreate the tree and the network.

In [9]:
# wrmodel=wr.weirules(rule_learner='tree', use_weights=True)
# wrmodel.fit_tree(X=attributes_train,
#              Y=class_labels_train,
#              en_classes=np.array(classes),
#              rule_columns=attribute_names_characteristics,
#              forest=True,
#              max_depth=30)
# wrmodel.create_network(rho=5.4)
# # save the model
# with open('./models/weirulesV6c-yahoo-r54-f.pkl', 'wb') as output:
#      pickle.dump(wrmodel, output, pickle.HIGHEST_PROTOCOL)

Load pickled version of the model.

In [10]:
with open('./models/weirulesV6c-yahoo-r54-f.pkl', 'rb') as input:
     wrmodel = pickle.load(input)

In [11]:
print(wrmodel.model)

Network(
  (conv1): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (conv2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (dropout2d_1): Dropout2d(p=0.2, inplace=False)
  (batchnorm2d1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2d2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (c_hidden_1): Linear(in_features=12544, out_features=1024, bias=True)
  (batchnorm1d_c1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Linear(in_features=1024, out_features=512, bias=True)
  (batchnorm1d_4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (merge_hidden): Linear(in_features=512, out_features=274, bias=True)
  (ors): ModuleList(
    (0): minmax(in_features=5, out_features=1, bias=False)
    (1): minmax(in_features=3, out_features=1, bias=False)
    (2): minmax(in_features=3, out_features=1, bias=False)
    (3): 

## Faster R-CNN training

Create the training and testing dataset instances to use in data loaders for Faster R-CNN

In [12]:
image_folder='sample/yahoo/ayahoo_test_images/'

#train
index_train=pd.DataFrame(image_paths_train.index,index=image_paths_train.index,columns=['index'])
dataset_train=pd.concat([index_train,image_paths_train,boxes_train,attributes_train], axis=1)
dataset_train.to_csv('sample/yahoo/train_data.csv', sep=' ')
yahoo_dataset_train_set = wutils.YahooDataset(dataset_train.values, class_labels_train.values, 6, image_folder)

#test
index_test=pd.DataFrame(image_paths_test.index,index=image_paths_test.index,columns=['index'])
dataset_test=pd.concat([index_test,image_paths_test,boxes_test,attributes_test], axis=1)
dataset_test.to_csv('sample/yahoo/test_data.csv', sep=' ')
yahoo_dataset_test_set = wutils.YahooDataset(dataset_test.values, class_labels_test.values, 6, image_folder)

Set the training parameters.

In [13]:
def collate_fn(batch):
    return tuple(zip(*batch))

# Parameters
params = {
        'batch_size': 25, 
        'shuffle': True,
        'num_workers': 0,
        'collate_fn': collate_fn,
        }

max_epochs=100
training_generator = data.DataLoader(yahoo_dataset_train_set, **params)
testing_generator = data.DataLoader(yahoo_dataset_test_set, **params)

Train Faster R-CNN with resnet50 backbone. (Uncomment if you want to retrain)

In [14]:
backbone = fw.resnet_fpn_backbone('resnet50', pretrained=True)
model = fw.FasterRCNN_Weirules(backbone, len(classes)+1)
model = model.cuda()

#frcnn_optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0005)
#wutils.train_frcnn(model, training_generator, frcnn_optimizer, max_epochs)

#torch.save(model.state_dict(), './models/frcnn_100_epochs_yahoo.model')

Load saved model trained with the above parameters.

In [15]:
model.load_state_dict(torch.load('./models/frcnn_100_epochs_yahoo.model'))

<All keys matched successfully>

### Deep feature extraction

Extract the corresponding deep features for the training and test set. (Uncomment if you want to re-extract them)

In [16]:
#deep_features_dict_train=wutils.extract_df(model,training_generator)
#with open('models/df_frcnn_100e_yahoo_train.pickle', 'wb') as output:
#       pickle.dump(deep_features_dict_train, output, pickle.HIGHEST_PROTOCOL)

#deep_features_dict_test=wutils.extract_df(model,testing_generator)
#with open('models/df_frcnn_100e_yahoo_test.pickle', 'wb') as output:
#      pickle.dump(deep_features_dict_test, output, pickle.HIGHEST_PROTOCOL)

Load pickled extracted deep features dict. The keys of the dictionary use the index column of the dataset, while the values include the extracted deep feature.

In [17]:
deep_features_dict_train={}
with open('models/df_frcnn_100e_yahoo_train.pickle', 'rb') as handle:
    deep_features_dict_train=pickle.load(handle)
deep_features_dict_test={}
with open('models/df_frcnn_100e_yahoo_test.pickle', 'rb') as handle:
    deep_features_dict_test=pickle.load(handle)

### WeiRules training

Create dataset instances that include the deep feature dicts.

In [18]:
yahoo_dataset_train_set_wr = wutils.YahooDataset(dataset_train.values, class_labels_train.values, 6, image_folder,deep_features_dict_train)
yahoo_dataset_test_set_wr = wutils.YahooDataset(dataset_test.values, class_labels_test.values, 6, image_folder,deep_features_dict_test)

Set the training parameters and the data loaders.

In [19]:
# Parameters
params_wr = {
        'batch_size': 64,
        'shuffle': True,
        'num_workers': 4,
        'collate_fn': collate_fn,
        }

max_epochs=100
training_generator_wr = data.DataLoader(yahoo_dataset_train_set_wr, **params_wr)
test_generator_wr = data.DataLoader(yahoo_dataset_test_set_wr, **params_wr)

Train the WeiRules network.

In [20]:
trained_wrmodel_path='models/weirulesV6c-yahoo-r54-f.model'
#weirules_optimizer = torch.optim.Adam(wrmodel.model.parameters(), lr=0.0001)
#losses=wutils.train_weirules(wrmodel, training_generator_wr, weirules_optimizer, max_epochs)
#torch.save(wrmodel.model.state_dict(), trained_wrmodel_path)

In [21]:
wrmodel.load_model(trained_wrmodel_path)

## Evaluation results

### WeiRules

Example of a fuzzified rule. Comparisons have been replaced with the appropriate form of the sigmoidal membership function, while conjunctions/disjunctions are approximated using the (weighted) exponential mean approximation.

In [22]:
print(wrmodel.get_fuzzified_rule_str(3))

IF
	0.27915024757385254 * (Wing <= 0.5 AND' Skin <= 0.5 AND' Hair > 0.5 AND' Furry <= 0.5)
	 OR' 0.27915024757385254 * (Wing <= 0.5 AND' Skin > 0.5 AND' Nose <= 0.5 AND' Eye > 0.5)
	 OR' 0.27915024757385254 * (Wing > 0.5)
THEN 3


In [23]:
all_best_results=wutils.predict_weirules(wrmodel, test_generator_wr)

In [24]:
wlabel=[]
alabel=[]
for res in all_best_results:
    wlabel.append(res[0])
    alabel.append(res[1])

In [25]:
print(classification_report(alabel,wlabel))

              precision    recall  f1-score   support

           0       0.93      0.78      0.85       137
           1       0.90      0.80      0.85       104
           2       0.83      0.56      0.67        72
           3       1.00      0.67      0.80        24
           4       0.72      0.33      0.46        63
           5       0.90      0.77      0.83        73
           6       0.98      0.97      0.98       179
           7       0.93      0.97      0.95        76
           8       0.58      0.99      0.73       103
           9       0.98      0.90      0.94        94
          10       0.50      0.70      0.58        84
          11       0.64      0.69      0.66        88

    accuracy                           0.80      1097
   macro avg       0.82      0.76      0.77      1097
weighted avg       0.83      0.80      0.80      1097



### Faster R-CNN

It would be more sensible to use an object detection metric such mAP for Faster R-CNN, since each image has one class we can assume take the prediction with the highest score.

In [26]:
# Parameters
params = {
        'batch_size': 1, # change to higher number for faster results if enough memory is available
        'shuffle': True,
        'num_workers': 0,
        'collate_fn': collate_fn,
        }

testing_generator = data.DataLoader(yahoo_dataset_test_set, **params)
all_best_results=wutils.predict_frcnn(model, testing_generator)


In [27]:
flabel = []
alabel = []
for res in all_best_results[0]:
    flabel.append(res[1]['label']-1)
    alabel.append(res[2])

print(classification_report(alabel, flabel))

              precision    recall  f1-score   support

           0       0.81      0.82      0.82       136
           1       0.85      0.91      0.88       102
           2       0.78      0.86      0.82        72
           3       0.29      0.42      0.34        24
           4       0.51      0.71      0.59        63
           5       0.66      0.53      0.59        73
           6       0.97      0.94      0.95       179
           7       0.66      0.64      0.65        76
           8       0.85      0.86      0.86       103
           9       0.80      0.59      0.68        93
          10       0.74      0.67      0.70        83
          11       0.96      0.91      0.94        88

    accuracy                           0.79      1092
   macro avg       0.74      0.74      0.73      1092
weighted avg       0.80      0.79      0.79      1092



## Decision tree

Decision tree with no hyper parameter optimisation.

In [28]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='gini')

clf = clf.fit(attributes_train, class_labels_train)
preds=clf.predict(attributes_test)
print(classification_report(class_labels_test, preds))

              precision    recall  f1-score   support

           0       0.56      0.98      0.71       137
           1       0.89      0.65      0.76       104
           2       1.00      0.31      0.47        72
           3       0.95      0.75      0.84        24
           4       0.69      0.32      0.43        63
           5       0.92      0.78      0.84        73
           6       1.00      0.96      0.98       179
           7       0.96      1.00      0.98        76
           8       0.94      0.91      0.93       103
           9       0.97      0.90      0.93        94
          10       0.43      0.67      0.52        84
          11       0.48      0.44      0.46        88

    accuracy                           0.77      1097
   macro avg       0.82      0.72      0.74      1097
weighted avg       0.81      0.77      0.76      1097

